In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [03:56<00:00,  7.57it/s]  


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:12<00:00, 80.81it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [8]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [9]:
trainer.train()

epoch=0.126953125, loss=5.051429271697998:   2%|▏         | 65/4096 [00:15<49:05,  1.37it/s] 

epoch=0.125, eval_loss=4.977377772331238


epoch=0.251953125, loss=4.219050884246826:   3%|▎         | 129/4096 [00:30<49:23,  1.34it/s] 

epoch=0.25, eval_loss=4.1699260622262955


epoch=0.376953125, loss=3.9263792037963867:   5%|▍         | 193/4096 [00:44<48:03,  1.35it/s]

...

epoch=7.75, eval_loss=3.167697347700596


epoch=7.876953125, loss=3.1172213554382324:  98%|█████████▊| 4033/4096 [16:33<00:46,  1.36it/s]

epoch=7.875, eval_loss=3.166757471859455


epoch=8.0, loss=3.0846242904663086: 100%|██████████| 4096/4096 [16:48<00:00,  4.06it/s]        

epoch=8.0, eval_loss=3.1591607332229614


# Оцениваем качество и проверяем жадную и случайную генерацию

In [10]:
trainer.evaluate()

3.1591607332229614

In [11]:
generate(model, tokenizer, temperature=0)

'Козерогам стоит быть внимательнее к своему здоровью. В конце дня вы не сможете выяснить отношения, то можете стать жертвой в связи с ними, не теряйтесь, если вы не будете снизить, если вы не будете отлично понять свои личные потребности или неверные шаги, не терять времени на будущее. В конце дня возрастает вероятность того, что вы не захотите отложить на вечер, так как вполне естественное внимание на будущее или накопленное слово и способность несколько неверно понять, если вы не будете отлично понять окружающих, то сейчас вы не сможете выяснить отношения, то можете стать жертвой в связи с ними, не теряйтесь, если вы не будете думать о своих чувствах и выяснить свои силы и настойчивые шаги в вашу жизнь. В конце дня у вас могут появиться новые знакомства, которые вы не готовы в тупик, то можете столкнуться с ними имиджом. В конце дня вы не сможете выяснить отношения, то можете стать жертвой в связи с ними, не теряйтесь, если вы не будете снизить, если вы не будете отлично понять свои 

In [12]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Прекрасный день для гармонизации отношений с окружающим миром и хозяйственной почвами для развития бизнеса, рекламных мероприятий, особенно для заключения сделок и договоров, открытия предприятий и начала лечения во благотворительных и обретения. Хороший день для любых начинаний, часто и открытия предприятий, дальней поездки, учебы, бизнесмены, а также для выступлений на новой культуры, искусством, искусством, подписания контрактов и выступлений и поездков, переговоров со спонсором и культурными органами, а также для заключения сделок, для выступлений и различных инструментов, а также для общественного деятеля, публичных выступлений. Полезно использовать для получения новых дел, особенно касающихся семейных отношений и методологических мероприятий. Вероятны конфликты с людьми издалека.'